# Universal $\mathcal{P}_1(\mathbb{R})$-Deep Neural Model (Type A)
---

---
# Training Algorithm:
---
## 1) Generate Data:
Generates the empirical measure $\sum_{n=1}^N \delta_{X_T(\omega_n)}$ of $X_T$ conditional on $X_0=x_0\in \mathbb{R}$ *($x_0$ and $T>0$ are user-provided)* by simulating from:
$$ 
X_T = x + \int_0^T \alpha(s,x)ds + \int_0^T(\beta(s,x)+\sigma_s^H)dW_s.
$$

## 2) Get "Sample Barycenters":
Let $\{\mu_n\}_{n=1}^N\subset\mathcal{P}_1(\mathbb{R}^d)$.  Then, the *sample barycenter* is defined by:
1. $\mathcal{M}^{(0)}\triangleq \left\{\hat{\mu}_n\right\}_{n=1}^N$,
2. For $1\leq n\leq \mbox{N sample barycenters}$: 
    - $
\mu^{\star}\in \underset{\tilde{\mu}\in \mathcal{M}^{(n)}}{\operatorname{argmin}}\, \sum_{n=1}^N \mathcal{W}_1\left(\mu^{\star},\mu_n\right),
$
    - $\mathcal{M}^{(n)}\triangleq \mathcal{M}^{(n-1)} - \{\mu^{\star}\},$
*i.e., the closest generated measure form the random sample to all other elements of the random sample.*

---
**Note:** *We simplify the computational burden of getting the correct classes by putting this right into this next loop.*

## 3) Train Deep Classifier:
$\hat{f}\in \operatorname{argmin}_{f \in \mathcal{NN}_{d:N}^{\star}} 
\sum_{x \in \mathbb{X}}
\, 
\mathbb{H}
\left(
    \operatorname{Softmax}_N\circ f(x)_n| I\left\{W_1(\hat{\mu}_n,\mu_x),\inf_{m\leq N} W_1(\hat{\mu}_m,\mu_x)\right\}
\right);
$
where $\mathbb{H}$ is the categorical cross-entropy.  

---
---
---
## Notes - Why the procedure is so computationally efficient?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

## Meta-Parameters

### Visualization

In [1]:
# How many random polulations to visualize:
Visualization_Size = 4

### Quantization
*This hyperparameter describes the proportion of the data used as sample-barycenters.*

In [2]:
Quantization_Proportion = 0.5

### Simulation

In [3]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 500
N_Monte_Carlo_Samples = 10*6
N_Monte_Carlo_Samples_Test = 10*4 # How many MC-samples to draw from test-set?

# Roughness
Rougness = 0.9
Ratio_fBM_to_typical_vol = 0

T_end = 1
T_end_test = 2
rough_SDE = True #This hyperparameter determines if we use a Euler-Maryama scheme or if we use something else.  

## Grid
N_Grid_Finess =3
Max_Grid = 1

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

#### Mode: Code-Testin Parameter(s)

In [4]:
trial_run = True

### Meta-parameters

In [5]:
# Test-size Ratio
test_size_ratio = .75

## SDE Simulation Hyper-Parameter(s)

### Drift

In [6]:
def alpha(t,x):
    return t*np.sin(math.pi*x) #+ np.exp(-t)

### Volatility

In [7]:
def beta(t,x):
    return (1+t) + np.cos(x)

## Semi-Martingale

In [8]:
def field_dirction_x(x):
    return x*np.cos(x)

def finite_variation_t(t):
    return t*(np.sin(math.pi*t) + np.exp(-t))

### Get Paths

In [9]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

### Import

In [10]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Load Helper Function(s)
# %run ParaGAN_Backend.ipynb
exec(open('Helper_Functions.py').read())
# Import time separately
import time

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready


### Set Seed

In [11]:
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

## Get Internal (Hyper)-Parameter(s)
*Initialize the hyperparameters which are fully-specified by the user-provided hyperparameter(s).*

### Initialize Grid
This is $\mathbb{X}$ and it represents the grid of initial states.

In [12]:
# Get Input Data
#----------------------------------------------------------#
## Train
x_Grid = np.arange(start=-Max_Grid,
                   stop=Max_Grid,
                   step=(2*Max_Grid/N_Grid_Finess))
t_Grid = np.linspace(0,T_end,(1+N_Euler_Maruyama_Steps))
## Get Number of Instances in Grid: Training
N_Grid_Instances_x = len(x_Grid)
N_Grid_Instances_t = len(t_Grid)
N_Grid_Instances = N_Grid_Instances_x*N_Grid_Instances_t 

#----------------------------------------------------------#
## Test
x_Grid_test = np.sort(np.random.uniform(low=-Max_Grid,
                                        high=Max_Grid,
                                        size = round(N_Grid_Instances*test_size_ratio)))
t_Grid_test = np.linspace(T_end+0.001,T_end_test,(1+round(N_Euler_Maruyama_Steps*test_size_ratio)))
# Get Number of Instances in Grid: Test
N_Grid_Instances_x_test = len(x_Grid_test)
N_Grid_Instances_t_test = len(t_Grid_test)
N_Grid_Instances_test = N_Grid_Instances_x_test*N_Grid_Instances_t_test
#----------------------------------------------------------#

# Updater User
print("\u2022 Grid Instances: ", N_Grid_Instances, "and :",N_Grid_Instances_test," Testing instances.")

• Grid Instances:  1503 and : 423752  Testing instances.


### Initialize Counting Parameters
Initialize the "conting" type parameters which will help us to determine the length of loops and to intialize object's size later on.  

In [13]:
# Get Internal (Counting) Parameters
N_Quantizers_to_parameterize = round(Quantization_Proportion*N_Grid_Instances)
N_Elements_Per_Cluster = int(round(N_Grid_Instances/N_Quantizers_to_parameterize))

# Update User
print("\u2022",N_Quantizers_to_parameterize," Centers will be produced; from a total datasize of: ",N_Grid_Finess,
      "!  (That's ",Quantization_Proportion,
      " percent).")
print("\u2022 Each Wasserstein-1 Ball should contain: ",
      N_Elements_Per_Cluster, 
      "elements from the training set.")

• 752  Centers will be produced; from a total datasize of:  3 !  (That's  0.5  percent).
• Each Wasserstein-1 Ball should contain:  2 elements from the training set.


---

### Simulate from non-Markovian SDE with rough volatility:
$d X_t = \alpha(t,X_t)dt + ((1-\eta)\beta(t,X_t)+\eta\sigma_t^H)dW_t ;\qquad X_0 =x$
Where $(\sigma_t^H)_t$ is a fBM with Hurst parameter $H=0.01$ and $\eta \in [0,1]$ controlls the 'abount of long-term memory and roughness in $X_t$'.

### Define Sampler - Data-Generator

Generates the empirical measure $\sum_{n=1}^N \delta_{X_T(\omega_n)}$ of $X_T$ conditional on $X_0=x_0\in \mathbb{R}$ *($x_0$ and $T>0$ are user-provided)*.

In [14]:
def Euler_Maruyama_Generator(x_0,
                             N_Euler_Maruyama_Steps = 10,
                             N_Monte_Carlo_Samples = 100,
                             T = 1,
                             Hurst = 0.1,
                             Ratio_fBM_to_typical_vol = 0.5): 
    
    #----------------------------#    
    # DEFINE INTERNAL PARAMETERS #
    #----------------------------#
    # Initialize Empirical Measure
    X_T_Empirical = np.zeros([N_Euler_Maruyama_Steps,N_Monte_Carlo_Samples])


    # Internal Initialization(s)
    ## Initialize current state
    n_sample = 0
    ## Initialize Incriments
    dt = T/N_Euler_Maruyama_Steps
    sqrt_dt = np.sqrt(dt)

    #-----------------------------#    
    # Generate Monte-Carlo Sample #
    #-----------------------------#
    while n_sample < N_Monte_Carlo_Samples:
        # Reset Step Counter
        t = 1
        # Initialize Current State 
        X_current = x_0
        # Generate roughness
        sigma_rough = FBM(n=N_Euler_Maruyama_Steps, hurst=0.75, length=1, method='daviesharte').fbm()
        # Perform Euler-Maruyama Simulation
        while t<(N_Euler_Maruyama_Steps-1):
            # Update Internal Parameters
            ## Get Current Time
            t_current = t*(T/N_Euler_Maruyama_Steps)

            # Update Generated Path
            drift_t = alpha(t_current,X_current)*dt
            vol_t = ((1-Ratio_fBM_to_typical_vol)*beta(t_current,X_current)+Ratio_fBM_to_typical_vol*(sigma_rough[t]))*np.random.normal(0,sqrt_dt)
            X_current = X_current + drift_t + vol_t

            # Update Counter (EM)
            t = t+1

            # Update Empirical Measure
            X_T_Empirical[t,n_sample] = X_current

        # Update Counter (MC)
        n_sample = n_sample + 1

    return X_T_Empirical

---

### Initializations

In [15]:
# Initialize List of Barycenters
Wasserstein_Barycenters = []
# Initialize Terminal-Time Empirical Measures
## Training Outputs
measures_locations_list = []
measures_weights_list = []
## Testing Outputs
measures_locations_test_list = []
measures_weights_test_list = []
# Grid (Training and Testing inputs (t,x))
X_train = []
X_test = []

# Initialize (Empirical) Weight(s)
measure_weights = np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples
measure_weights_test = np.ones(N_Monte_Carlo_Samples_Test)/N_Monte_Carlo_Samples_Test
# Initialize Quantizer
Init_Quantizer_generic = np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples

## Generate $\{\hat{\nu}^{N}_{T,x}\}_{x \in \mathbb{X}}$ Build Wasserstein Cover

#### Get Data

In [16]:
# Update User
print("Current Monte-Carlo Step:")
if rough_SDE == False:
    print("===================================")
    print("Start Simulation Step: Training Set")
    print("===================================")
    # Initialize fBM Generator
    fBM_Generator = FBM(n=N_Euler_Maruyama_Steps, hurst=0.75, length=1, method='daviesharte')

    # Perform Monte-Carlo Data Generation
    for i in tqdm(range(N_Grid_Instances_x)):
        # Get x
        field_loop_x = field_dirction_x(x_Grid[i])
        # Get omega and t
        # Generate finite-variation path (since it stays unchanged)
        finite_variation_path = finite_variation_t(t_Grid).reshape(-1,1) +field_loop_x
        for n_MC in range(N_Monte_Carlo_Samples):
            fBM_variation_path_loop = fBM_Generator.fbm().reshape(-1,1)
            generated_path_loop = finite_variation_path + fBM_variation_path_loop
            if n_MC == 0:
                paths_loop = generated_path_loop
            else:
                paths_loop = np.append(paths_loop,generated_path_loop,axis=-1)
        
        # Map numpy to list
        measures_locations_loop = paths_loop.tolist()
        # Get inputs
        X_train_loop = np.append(np.repeat(x_Grid[i],(N_Euler_Maruyama_Steps+1)).reshape(-1,1),
                                 t_Grid.reshape(-1,1),
                                 axis=1)
        
        # Append to List
        measures_locations_list = measures_locations_list + measures_locations_loop
        measures_weights_list.append(measure_weights)
        
        # Update Inputs
        if i==0:
            X_train = X_train_loop
        else:
            X_train = np.append(X_train,X_train_loop,axis=0)
    
    # Update User
    print("==================================")
    print("Done Simulation Step: Training Set")
    print("==================================")


print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")

if rough_SDE == False:
    print("===============================")
    print("Start Simulation Step: Test Set")
    print("===============================")
    # Initialize fBM Generator
    fBM_Generator_test = FBM(n=(len(t_Grid_test)-1), hurst=0.75, length=1, method='daviesharte')

    # Perform Monte-Carlo Data Generation
    for i in tqdm(range(N_Grid_Instances_x_test)):
        # Get x
        field_loop_x = field_dirction_x(x_Grid_test[i])
        # Get omega and t
        # Generate finite-variation path (since it stays unchanged)
        finite_variation_path = finite_variation_t(t_Grid_test).reshape(-1,1) +field_loop_x
        for n_MC in range(N_Monte_Carlo_Samples_Test):
            fBM_variation_path_loop = fBM_Generator_test.fbm().reshape(-1,1)
            generated_path_loop = finite_variation_path + fBM_variation_path_loop
            if n_MC == 0:
                paths_loop = generated_path_loop
            else:
                paths_loop = np.append(paths_loop,generated_path_loop,axis=-1)
        
        # Map numpy to list
        measures_locations_loop = paths_loop.tolist()
        # Get inputs
        X_test_loop = np.append(np.repeat(x_Grid_test[i],len(t_Grid_test)).reshape(-1,1),
                                 t_Grid_test.reshape(-1,1),
                                 axis=1)
        
        # Append to List
        measures_locations_test_list = measures_locations_test_list + measures_locations_loop
        measures_weights_test_list.append(measure_weights_test)
        
        # Update Inputs
        if i==0:
            X_test = X_test_loop
        else:
            X_test = np.append(X_test,X_train_loop,axis=0)
    print("==============================")
    print("Done Simulation Step: Test Set")
    print("==============================")
    
print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")

Current Monte-Carlo Step:
===============================--------------------------------------===============================
===============================--------------------------------------===============================
===============================--------------------------------------===============================
===============================--------------------------------------===============================
===============================--------------------------------------===============================
===============================--------------------------------------===============================


In [17]:
if rough_SDE == True:
    print("Using Euler-Maruyama distritization + Monte-Carlo Sampling.")
    #----------------------------------------------------------------------------------------------#
    # Update User
    print("===================================")
    print("Start Simulation Step: Training Set")
    print("===================================")
    # Initialize fBM Generator
    fBM_Generator = FBM(n=N_Euler_Maruyama_Steps, hurst=0.75, length=1, method='daviesharte')

    # Perform Monte-Carlo Data Generation
    for i in tqdm(range(N_Grid_Instances_x)):
        # Get x
        field_loop_x = field_dirction_x(x_Grid[i])
        # Get omega and t
        # Generate finite-variation path (since it stays unchanged)
        finite_variation_path = finite_variation_t(t_Grid).reshape(-1,1) +field_loop_x
        # Simulate Paths
        paths_loop = Euler_Maruyama_Generator(x_0=x_Grid[i],
                                              N_Euler_Maruyama_Steps = len(t_Grid),
                                              N_Monte_Carlo_Samples = N_Monte_Carlo_Samples,
                                              T = T_end,
                                              Hurst=Rougness,
                                              Ratio_fBM_to_typical_vol=Ratio_fBM_to_typical_vol)
        
        # Map numpy to list
        measures_locations_loop = paths_loop.tolist()
        # Get inputs
        X_train_loop = np.append(np.repeat(x_Grid[i],(N_Euler_Maruyama_Steps+1)).reshape(-1,1),
                                 t_Grid.reshape(-1,1),
                                 axis=1)
        
        # Append to List
        measures_locations_list = measures_locations_list + measures_locations_loop
        measures_weights_list.append(measure_weights)
        
        # Update Inputs
        if i==0:
            X_train = X_train_loop
        else:
            X_train = np.append(X_train,X_train_loop,axis=0)
    
    # Update User
    print("==================================")
    print("Done Simulation Step: Training Set")
    print("==================================")


print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")
print("===============================--------------------------------------===============================")

if rough_SDE == True:
    print("===============================")
    print("Start Simulation Step: Test Set")
    print("===============================")
    # Initialize fBM Generator
    fBM_Generator_test = FBM(n=(len(t_Grid_test)-1), hurst=0.75, length=1, method='daviesharte')

    # Perform Monte-Carlo Data Generation
    for i in tqdm(range(N_Grid_Instances_x_test)):
        # Get x
        field_loop_x = field_dirction_x(x_Grid_test[i])
        # Get omega and t
        # Generate finite-variation path (since it stays unchanged)
        finite_variation_path = finite_variation_t(t_Grid_test).reshape(-1,1) +field_loop_x
        paths_loop = Euler_Maruyama_Generator(x_0=x_Grid_test[i],
                                              N_Euler_Maruyama_Steps = len(t_Grid_test),
                                              N_Monte_Carlo_Samples = N_Monte_Carlo_Samples_Test,
                                              T = T_end_test,
                                              Hurst=Rougness,
                                              Ratio_fBM_to_typical_vol=Ratio_fBM_to_typical_vol)
        
        # Map numpy to list
        measures_locations_loop = paths_loop.tolist()
        # Get inputs
        X_test_loop = np.append(np.repeat(x_Grid_test[i],len(t_Grid_test)).reshape(-1,1),
                                 t_Grid_test.reshape(-1,1),
                                 axis=1)
        
        # Append to List
        measures_locations_test_list = measures_locations_test_list + measures_locations_loop
        measures_weights_test_list.append(measure_weights_test)
        
        # Update Inputs
        if i==0:
            X_test = X_test_loop
        else:
            X_test = np.append(X_test,X_train_loop,axis=0)
    print("==============================")
    print("Done Simulation Step: Test Set")
    print("==============================")

  0%|          | 0/3 [00:00<?, ?it/s]

Using Euler-Maruyama distritization + Monte-Carlo Sampling.
Start Simulation Step: Training Set


  0%|          | 0/1127 [00:00<?, ?it/s]

Done Simulation Step: Training Set
===============================--------------------------------------===============================
===============================--------------------------------------===============================
===============================--------------------------------------===============================
Start Simulation Step: Test Set


100%|██████████| 1127/1127 [07:54<00:00,  2.37it/s]

Done Simulation Step: Test Set


#### Start Timer (Model Type A)

In [18]:
# Start Timer
Type_A_timer_Begin = time.time()

## Get "Sample Barycenters":
Let $\{\mu_n\}_{n=1}^N\subset\mathcal{P}_1(\mathbb{R}^d)$.  Then, the *sample barycenter* is defined by:
1. $\mathcal{M}^{(0)}\triangleq \left\{\hat{\mu}_n\right\}_{n=1}^N$,
2. For $1\leq n\leq \mbox{N sample barycenters}$: 
    - $
\mu^{\star}\in \underset{\tilde{\mu}\in \mathcal{M}^{(n)}}{\operatorname{argmin}}\, \sum_{n=1}^N \mathcal{W}_1\left(\mu^{\star},\mu_n\right),
$
    - $\mathcal{M}^{(n)}\triangleq \mathcal{M}^{(n-1)} - \{\mu^{\star}\},$
*i.e., the closest generated measure form the random sample to all other elements of the random sample.*

---
**Note:** *We simplify the computational burden of getting the correct classes by putting this right into this next loop.*

---

## Build Dissimilarity (Distance) Matrix
*In this step we build a dissimularity matrix of the dataset on the Wasserstein-1 space.  Namely:*
$$
\operatorname{Mat}_{\# \mathbb{X},\# \mathbb{X}}\left(\mathbb{R}\right)\ni D; \text{ where}\qquad \, D_{i,j}\triangleq \mathcal{W}_1\left(f(x_i),f(x_j)\right)
;
$$
*where $f\in C\left((\mathcal{X},\mathcal{P}_1(\mathcal{Y})\right)$ is the "target" function we are learning.*

**Note**: *Computing the dissimularity matrix is the most costly part of the entire algorithm with a complexity of at-most $\mathcal{O}\left(E_{W} \# \mathbb{X})^2\right)$ where $E_W$ denotes the complexity of a single Wasserstein-1 evaluation between two elements of the dataset.*

In [19]:
# Initialize Disimilarity Matrix
Dissimilarity_matrix_ot = np.zeros([N_Grid_Instances,N_Grid_Instances])


# Update User
print("\U0001F61A"," Begin Building Distance Matrix"," \U0001F61A")
# Build Disimilarity Matrix
for i in tqdm(range(N_Grid_Instances)):
    for j in range(N_Grid_Instances):
        Dissimilarity_matrix_ot[i,j] = ot.emd2_1d(measures_locations_list[j],
                                                  measures_locations_list[i])
# Update User
print("\U0001F600"," Done Building Distance Matrix","\U0001F600","!")

  0%|          | 0/1503 [00:00<?, ?it/s]

😚  Begin Building Distance Matrix  😚


100%|██████████| 1503/1503 [04:32<00:00,  5.51it/s]

😀  Done Building Distance Matrix 😀 !


## Initialize Quantities to Loop Over

## Get "Sample Barycenters" and Generate Classes

In [20]:
# Initialize Locations Matrix (Internal to Loop)
measures_locations_list_current = copy.copy(measures_locations_list)
Dissimilarity_matrix_ot_current = copy.copy(Dissimilarity_matrix_ot)

# Initialize masker vector
masker = np.ones(N_Grid_Instances)

# Initialize Sorting Reference Vector (This helps us efficiently scroll through the disimularity matrix to identify the barycenter without having to re-compute the dissimultarity matrix of a sub-saple at every iteration (which is the most costly part of the algorithm!))
Distances_Loop = Dissimilarity_matrix_ot_current.sum(axis=1)

# Initialize Classes (In-Sample)
Classifer_Wasserstein_Centers = np.zeros([N_Quantizers_to_parameterize,N_Grid_Instances])

In [21]:
# Update User
print("\U0001F61A"," Begin Identifying Sample Barycenters"," \U0001F61A")

# Identify Sample Barycenters
for i in tqdm(range(N_Quantizers_to_parameterize)):    
    # GET BARYCENTER #
    #----------------#
    ## Identify row with minimum total distance
    Barycenter_index = int(Distances_Loop.argsort()[:1][0])
    ## Get Barycenter
    ## Update Barycenters Array ##
    #----------------------------#
    ### Get next Barycenter
    new_barycenter_loop = np.array(measures_locations_list_current[Barycenter_index]).reshape(-1,1)
    ### Update Array of Barycenters
    if i == 0:
        # Initialize Barycenters Array
        Barycenters_Array = new_barycenter_loop
    else:
        # Populate Barycenters Array
        Barycenters_Array = np.append(Barycenters_Array,new_barycenter_loop,axis=-1)

    # GET CLUSTER #
    #-------------#
    # Identify Cluster for this barycenter (which elements are closest to it)
    Cluster_indices = (masker*Dissimilarity_matrix_ot_current[:,Barycenter_index]).argsort()[:N_Elements_Per_Cluster]
    ## UPDATES Set  M^{(n)}  ##
    #-------------------------#
    Dissimilarity_matrix_ot_current[Cluster_indices,:] = 0
    # Distance-Based Sorting
    Distances_Loop[Cluster_indices] = math.inf

    # Update Cluster
    masker[Cluster_indices] = math.inf
    
    # Update Classes
    Classifer_Wasserstein_Centers[i,Cluster_indices] = 1
#     print(Cluster_indices)

# Update User
print("\U0001F600"," Done Identifying Sample Barycenters","\U0001F600","!")
print(Classifer_Wasserstein_Centers)

100%|██████████| 752/752 [00:00<00:00, 5271.14it/s]

😚  Begin Identifying Sample Barycenters  😚
😀  Done Identifying Sample Barycenters 😀 !
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


---

### Train Deep Classifier

In this step, we train a deep (feed-forward) classifier:
$$
\hat{f}\triangleq \operatorname{Softmax}_N\circ W_J\circ \sigma \bullet \dots \sigma \bullet W_1,
$$
to identify which barycenter we are closest to.

Re-Load Grid and Redefine Relevant Input/Output dimensions in dictionary.

#### Train Deep Classifier

In [22]:
# Re-Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Re-Load Classifier Function(s)
exec(open('Helper_Functions.py').read())

Deep Feature Builder - Ready
Deep Classifier - Ready


In [23]:
# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [2]
param_grid_Deep_Classifier['output_dim'] = [N_Quantizers_to_parameterize]

# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter = n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train, 
                                                                                                        y_train = Classifer_Wasserstein_Centers.T,
                                                                                                        X_test = X_test)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   53.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   53.2s finished


Epoch 1/400
47/47 [==============================] - 0s 3ms/step - loss: 6.6340 - accuracy: 0.0000e+00
Epoch 2/400
47/47 [==============================] - 0s 3ms/step - loss: 6.4363 - accuracy: 0.0020
Epoch 3/400
47/47 [==============================] - 0s 3ms/step - loss: 5.9583 - accuracy: 6.6534e-04
Epoch 4/400
47/47 [==============================] - 0s 3ms/step - loss: 5.5089 - accuracy: 0.0020
Epoch 5/400
47/47 [==============================] - 0s 4ms/step - loss: 5.1697 - accuracy: 0.0053
Epoch 6/400
47/47 [==============================] - 0s 4ms/step - loss: 4.8782 - accuracy: 0.0040
Epoch 7/400
47/47 [==============================] - 0s 4ms/step - loss: 4.6943 - accuracy: 0.0067
Epoch 8/400
47/47 [==============================] - 0s 4ms/step - loss: 4.5527 - accuracy: 0.0067
Epoch 9/400
47/47 [==============================] - 0s 3ms/step - loss: 4.4599 - accuracy: 0.0080
Epoch 10/400
47/47 [==============================] - 0s 3ms/step - loss: 4.3724 - accuracy: 0.0120
E

47/47 [==============================] - 0s 3ms/step - loss: 2.8976 - accuracy: 0.0625
Epoch 83/400
47/47 [==============================] - 0s 3ms/step - loss: 2.9034 - accuracy: 0.0539
Epoch 84/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8876 - accuracy: 0.0592
Epoch 85/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8755 - accuracy: 0.0605
Epoch 86/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8699 - accuracy: 0.0699
Epoch 87/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8492 - accuracy: 0.0632
Epoch 88/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8511 - accuracy: 0.0592: 0s - loss: 2.6922 - accuracy: 0.
Epoch 89/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8512 - accuracy: 0.0612
Epoch 90/400
47/47 [==============================] - 0s 3ms/step - loss: 2.8563 - accuracy: 0.0612
Epoch 91/400
47/47 [==============================] - 0s 3ms/step - loss: 2.833

47/47 [==============================] - 0s 3ms/step - loss: 2.4804 - accuracy: 0.0951
Epoch 164/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4470 - accuracy: 0.1005
Epoch 165/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4407 - accuracy: 0.1031
Epoch 166/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4349 - accuracy: 0.1045
Epoch 167/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4474 - accuracy: 0.0912
Epoch 168/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4447 - accuracy: 0.1045
Epoch 169/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4199 - accuracy: 0.0951
Epoch 170/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4449 - accuracy: 0.1084
Epoch 171/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4316 - accuracy: 0.0912
Epoch 172/400
47/47 [==============================] - 0s 3ms/step - loss: 2.4408 - accuracy: 0.1038
Epoc

47/47 [==============================] - 0s 3ms/step - loss: 2.2274 - accuracy: 0.1371
Epoch 245/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2372 - accuracy: 0.1324
Epoch 246/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2445 - accuracy: 0.1331
Epoch 247/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2358 - accuracy: 0.1344
Epoch 248/400
47/47 [==============================] - 0s 3ms/step - loss: 2.1978 - accuracy: 0.1277
Epoch 249/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2228 - accuracy: 0.1238
Epoch 250/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2303 - accuracy: 0.1430
Epoch 251/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2046 - accuracy: 0.1364
Epoch 252/400
47/47 [==============================] - 0s 4ms/step - loss: 2.2012 - accuracy: 0.1264
Epoch 253/400
47/47 [==============================] - 0s 3ms/step - loss: 2.2035 - accuracy: 0.1344
Epoc

47/47 [==============================] - 0s 3ms/step - loss: 2.1067 - accuracy: 0.1484
Epoch 326/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0611 - accuracy: 0.1703
Epoch 327/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0570 - accuracy: 0.1730
Epoch 328/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0686 - accuracy: 0.1557
Epoch 329/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0571 - accuracy: 0.1830
Epoch 330/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0574 - accuracy: 0.1657
Epoch 331/400
47/47 [==============================] - 0s 4ms/step - loss: 2.0853 - accuracy: 0.1597
Epoch 332/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0781 - accuracy: 0.1464
Epoch 333/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0492 - accuracy: 0.1750
Epoch 334/400
47/47 [==============================] - 0s 3ms/step - loss: 2.0516 - accuracy: 0.1610
Epoc

#### Get Predicted Quantized Distributions
- Each *row* of "Predicted_Weights" is the $\beta\in \Delta_N$.
- Each *Column* of "Barycenters_Array" denotes the $x_1,\dots,x_N$ making up the points of the corresponding empirical measures.

In [ ]:
# Format Weights
## Train
print("#---------------------------------------#")
print("Building Training Set (Regression): START")
print("#---------------------------------------#")
Predicted_Weights = np.array([])
for i in tqdm(range(N_Quantizers_to_parameterize)):    
    b = np.repeat(np.array(predicted_classes_train[:,i],dtype='float').reshape(-1,1),N_Monte_Carlo_Samples,axis=-1)
    b = b/N_Monte_Carlo_Samples
    if i ==0 :
        Predicted_Weights = b
    else:
        Predicted_Weights = np.append(Predicted_Weights,b,axis=1)
print("#-------------------------------------#")
print("Building Training Set (Regression): END")
print("#-------------------------------------#")

## Test
print("#-------------------------------------#")
print("Building Test Set (Predictions): START")
print("#-------------------------------------#")
Predicted_Weights_test = np.array([])
for i in tqdm(range(N_Quantizers_to_parameterize)):
    b_test = np.repeat(np.array(predicted_classes_test[:,i],dtype='float').reshape(-1,1),N_Monte_Carlo_Samples,axis=-1)
    b_test = b_test/N_Monte_Carlo_Samples
    if i ==0 :
        Predicted_Weights_test = b_test
    else:
        Predicted_Weights_test = np.append(Predicted_Weights_test,b_test,axis=1)
print("#-----------------------------------#")
print("Building Test Set (Predictions): END")
print("#-----------------------------------#")
        
# Format Points of Mass
print("#-----------------------------#")
print("Building Barycenters Set: START")
print("#-----------------------------#")
Barycenters_Array = Barycenters_Array.T.reshape(-1,)
print("#-----------------------------#")
print("Building Barycenters Set: END")
print("#-----------------------------#")

  4%|▍         | 29/752 [00:00<00:02, 289.63it/s]

#---------------------------------------#
Building Training Set (Regression): START
#---------------------------------------#


  0%|          | 0/752 [00:00<?, ?it/s]

#-------------------------------------#
Building Training Set (Regression): END
#-------------------------------------#
#-------------------------------------#
Building Test Set (Predictions): START
#-------------------------------------#


  5%|▍         | 34/752 [03:30<3:30:13, 17.57s/it]

#### Stop Timer

In [ ]:
# Stop Timer
Type_A_timer_end = time.time()
# Compute Lapsed Time Needed For Training
Time_Lapse_Model_A = Type_A_timer_end - Type_A_timer_Begin

## Get Moment Predictions

#### Write Predictions

### Training-Set Result(s): 

In [ ]:
print("Building Training Set Performance Metrics")

# Initialize Wasserstein-1 Error Distribution
W1_errors = np.array([])
Mean_errors = np.array([])
Var_errors = np.array([])
Skewness_errors = np.array([])
Kurtosis_errors = np.array([])
#---------------------------------------------------------------------------------------------#

# Populate Error Distribution
for x_i in tqdm(range(len(measures_locations_list)-1)):    
    # Get Laws
    W1_loop = ot.emd2_1d(Barycenters_Array,
                         np.array(measures_locations_list[x_i]).reshape(-1,),
                         Predicted_Weights[x_i,].reshape(-1,),
                         measure_weights.reshape(-1,))
    W1_errors = np.append(W1_errors,W1_loop)
    # Get Means
    Mu_hat = np.sum((Predicted_Weights[x_i])*(Barycenters_Array))
    Mu = np.mean(np.array(measures_locations_list[x_i]))
    Mean_errors =  np.append(Mean_errors,(Mu_hat-Mu))
    # Get Var (non-centered)
    Var_hat = np.sum((Barycenters_Array**2)*(Predicted_Weights[x_i]))
    Var = np.mean(np.array(measures_locations_list[x_i])**2)
    Var_errors = np.append(Var_errors,(Var_hat-Var)**2)
    # Get skewness (non-centered)
    Skewness_hat = np.sum((Barycenters_Array**3)*(Predicted_Weights[x_i]))
    Skewness = np.mean(np.array(measures_locations_list[x_i])**3)
    Skewness_errors = np.append(Skewness_errors,(abs(Skewness_hat-Skewness))**(1/3))
    # Get skewness (non-centered)
    Kurtosis_hat = np.sum((Barycenters_Array**4)*(Predicted_Weights[x_i]))
    Kurtosis = np.mean(np.array(measures_locations_list[x_i])**4)
    Kurtosis_errors = np.append(Kurtosis_errors,(abs(Kurtosis_hat-Kurtosis))**.25)
    
#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
W1_Performance = np.array([np.mean(np.abs(W1_errors)),np.mean(W1_errors**2)])
Mean_prediction_Performance = np.array([np.mean(np.abs(Mean_errors)),np.mean(Mean_errors**2)])
Var_prediction_Performance = np.array([np.mean(np.abs(Var_errors)),np.mean(Var_errors**2)])
Skewness_prediction_Performance = np.array([np.mean(np.abs(Skewness_errors)),np.mean(Skewness_errors**2)])
Kurtosis_prediction_Performance = np.array([np.mean(np.abs(Kurtosis_errors)),np.mean(Kurtosis_errors**2)])

Type_A_Prediction = pd.DataFrame({"W1":W1_Performance,
                                  "E[X']-E[X]":Mean_prediction_Performance,
                                  "(E[X'^2]-E[X^2])^.5":Var_prediction_Performance,
                                  "(E[X'^3]-E[X^3])^(1/3)":Skewness_prediction_Performance,
                                  "(E[X'^4]-E[X^4])^.25":Kurtosis_prediction_Performance},index=["MAE","MSE"])

# Write Performance
Type_A_Prediction.to_latex((results_tables_path+str("Roughness_")+str(Rougness)+str("__RatiofBM_")+str(Ratio_fBM_to_typical_vol)+
 "__TypeAPrediction_Train.tex"))


#---------------------------------------------------------------------------------------------#
# Update User
print(Type_A_Prediction)

---

### Test-Set Result(s): 

In [ ]:
print("Building Test Set Performance Metrics")

# Initialize Wasserstein-1 Error Distribution
W1_errors_test = np.array([])
Mean_errors_test = np.array([])
Var_errors_test = np.array([])
Skewness_errors_test = np.array([])
Kurtosis_errors_test = np.array([])
#---------------------------------------------------------------------------------------------#

# Populate Error Distribution
for x_i in tqdm(range(len(measures_locations_test_list)-1)):    
    # Get Laws
    W1_loop_test = ot.emd2_1d(Barycenters_Array,
                         np.array(measures_locations_test_list[x_i]).reshape(-1,),
                         Predicted_Weights_test[x_i,].reshape(-1,),
                         measure_weights_test.reshape(-1,))
    W1_errors_test = np.append(W1_errors_test,W1_loop_test)
    # Get Means
    Mu_hat_test = np.sum((Predicted_Weights_test[x_i])*(Barycenters_Array))
    Mu_test = np.mean(np.array(measures_locations_test_list[x_i]))
    Mean_errors_test =  np.append(Mean_errors_test,(Mu_hat_test-Mu_test))
    # Get Var (non-centered)
    Var_hat_test = np.sum((Barycenters_Array**2)*(Predicted_Weights_test[x_i]))
    Var_test = np.mean(np.array(measures_locations_test_list[x_i])**2)
    Var_errors_test = np.append(Var_errors_test,(Var_hat_test-Var_test)**2)
    # Get skewness (non-centered)
    Skewness_hat_test = np.sum((Barycenters_Array**3)*(Predicted_Weights_test[x_i]))
    Skewness_test = np.mean(np.array(measures_locations_test_list[x_i])**3)
    Skewness_errors_test = np.append(Skewness_errors_test,(abs(Skewness_hat_test-Skewness_test))**(1/3))
    # Get skewness (non-centered)
    Kurtosis_hat_test = np.sum((Barycenters_Array**4)*(Predicted_Weights_test[x_i]))
    Kurtosis_test = np.mean(np.array(measures_locations_test_list[x_i])**4)
    Kurtosis_errors_test = np.append(Kurtosis_errors_test,(abs(Kurtosis_hat_test-Kurtosis_test))**.25)
    
#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
W1_Performance_test = np.array([np.mean(np.abs(W1_errors_test)),np.mean(W1_errors_test**2)])
Mean_prediction_Performance_test = np.array([np.mean(np.abs(Mean_errors_test)),np.mean(Mean_errors_test**2)])
Var_prediction_Performance_test = np.array([np.mean(np.abs(Var_errors_test)),np.mean(Var_errors_test**2)])
Skewness_prediction_Performance_test = np.array([np.mean(np.abs(Skewness_errors_test)),np.mean(Skewness_errors_test**2)])
Kurtosis_prediction_Performance_test = np.array([np.mean(np.abs(Kurtosis_errors_test)),np.mean(Kurtosis_errors_test**2)])

Type_A_Prediction_test = pd.DataFrame({"W1":W1_Performance_test,
                                  "E[X']-E[X]":Mean_prediction_Performance_test,
                                  "(E[X'^2]-E[X^2])^.5":Var_prediction_Performance_test,
                                  "(E[X'^3]-E[X^3])^(1/3)":Skewness_prediction_Performance_test,
                                  "(E[X'^4]-E[X^4])^.25":Kurtosis_prediction_Performance_test},index=["MAE","MSE"])

# Write Performance
Type_A_Prediction_test.to_latex((results_tables_path+str("Roughness_")+str(Rougness)+str("__RatiofBM_")+str(Ratio_fBM_to_typical_vol)+
 "__TypeAPrediction_Test.tex"))

## Update User

### Training-Set Performance

In [ ]:
Type_A_Prediction

### Test-Set Performance

In [ ]:
Type_A_Prediction_test

### Print for Terminal Legibility

In [ ]:
print("#----------------------#")
print("Training-Set Performance")
print("#----------------------#")
print(Type_A_Prediction)
print(" ")
print(" ")
print(" ")

print("#------------------#")
print("Test-Set Performance")
print("#------------------#")
print(Type_A_Prediction_test)
print(" ")
print(" ")
print(" ")
# Update User
print("\u2022",N_Quantizers_to_parameterize," Centers were be produced; from a total datasize of: ",N_Grid_Finess,
      "!  (That's ",Quantization_Proportion,
      " percent).")
print("\u2022 Each Wasserstein-1 Ball should contain: ",
      N_Elements_Per_Cluster, 
      "elements from the training set.")

# Visualization of Performance
Randomly subsample from output space and visualize empirical measures!

In [ ]:
# # Adjust if is number of plots to visualizes is larger than number of output distributions (But only if there is not enough data!)
# if N_Grid_Instances <= Visualization_Size**2:
#         Visualization_Size = int(round(np.sqrt(min(N_Grid_Instances,Visualization_Size**2)))-1)


# # Initialize Random Sample of input-output pairs to visualize
# plotting_distribution_indices = random.sample(range(N_Grid_Instances), (Visualization_Size)**2)

# # Generate Plot
# f, axarr = plt.subplots(Visualization_Size,Visualization_Size,figsize=(6, 6), dpi=80, facecolor='w', edgecolor='k')
# plt.suptitle("Sample of Predictions")
# for i in range(Visualization_Size):
#     for j in range(Visualization_Size):
#         # Get Current (Randomly chosen (uniformly)) Index
#         current_index = (i*Visualization_Size + j)
#         current_random_index = plotting_distribution_indices[current_index]
#         # Generate Current Plot
#         axarr[i,j].bar(Barycenters_Array,(Predicted_Weights[current_random_index].reshape(-1,)), alpha=0.5,label="Prediction",color="chartreuse")
#         axarr[i,j].bar(measures_locations_list[current_random_index].reshape(-1,),measures_weights_list[current_random_index], alpha=0.5,label="Target",color="purple")

---

---
# Fin
---

---